# Many Models Forecasting (MMF)
This notebook showcases how to run MMF with local models on multiple univariate time series of weekly resolution. We will use [M4 competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128#sec5) data.

### Cluster setup

We recommend using a cluster with [Databricks Runtime 17.3 LTS for ML](https://docs.databricks.com/en/release-notes/runtime/17.3lts-ml.html). The cluster can be either a single-node or multi-node CPU cluster. MMF leverages [Pandas UDF](https://docs.databricks.com/en/udf/pandas.html) under the hood and utilizes all the available resource. Make sure to set the following Spark configurations before you start your cluster: [`spark.sql.execution.arrow.enabled true`](https://spark.apache.org/docs/3.0.1/sql-pyspark-pandas-with-arrow.html#enabling-for-conversion-tofrom-pandas) and [`spark.sql.adaptive.enabled false`](https://spark.apache.org/docs/latest/sql-performance-tuning.html#adaptive-query-execution). You can do this by specifying [Spark configuration](https://docs.databricks.com/en/compute/configure.html#spark-configuration) in the advanced options on the cluster creation page.

### Install and import packages
Check out [requirements.txt](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/requirements.txt) if you're interested in the libraries we use.

In [0]:
%pip install -r ../../requirements-local.txt --quiet
%pip install datasetsforecast==0.0.8 --quiet
dbutils.library.restartPython()

In [0]:
import pathlib
import pandas as pd
from datasetsforecast.m4 import M4
from mmf_sa import run_forecast

### Prepare data 
We are using [`datasetsforecast`](https://github.com/Nixtla/datasetsforecast/tree/main/) package to download M4 data. M4 dataset contains a set of time series which we use for testing MMF. Below we have written a number of custome functions to convert M4 time series to an expected format.

In [0]:
# Number of time series
n = 100


def create_m4_weekly():
    y_df, _, _ = M4.load(directory=str(pathlib.Path.home()), group="Weekly")
    target_ids = {f"W{i}" for i in range(1, n)}
    y_df = y_df[y_df["unique_id"].isin(target_ids)]
    y_df = (
        y_df.groupby("unique_id", group_keys=False)
             .apply(lambda g: transform_group(g, g.name))
             .reset_index(drop=True)
    )
    return y_df


def transform_group(df, unique_id):
    if len(df) > 260:
        df = df.iloc[-260:]
    start = pd.Timestamp("2020-01-01")
    date_idx = pd.date_range(start=start, periods=len(df), freq="W", name="ds")
    res_df = pd.DataFrame({
        "ds": date_idx,
        "unique_id": unique_id,
        "y": df["y"].to_numpy()
    })
    return res_df

We are going to save this data in a delta lake table. Provide catalog and database names where you want to store the data.

In [0]:
catalog = "mmf" # Name of the catalog we use to manage our assets
db = "m4" # Name of the schema we use to manage our assets (e.g. datasets)
user = spark.sql('select current_user() as user').collect()[0]['user'] # User email address

In [0]:
# Making sure that the catalog and the schema exist
_ = spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
_ = spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{db}")

(
    spark.createDataFrame(create_m4_weekly())
    .write.format("delta").mode("overwrite")
    .saveAsTable(f"{catalog}.{db}.m4_weekly_train")
)

Let's take a peak at the dataset:

In [0]:
display(
  spark.sql(f"select * from {catalog}.{db}.m4_weekly_train where unique_id in ('W1', 'W2', 'W3', 'W4', 'W5') order by unique_id, ds")
  )

If the number of time series is larger than the number of total cores, we set `spark.sql.shuffle.partitions` to the number of cores (can also be a multiple) so that we don't under-utilize the resource.

In [0]:
# Get the current value of shuffle partitions
current = spark.conf.get("spark.sql.shuffle.partitions")

# If not set to 'auto' (serverless), convert to int; otherwise, use default 200
if current != "auto":
    current_val = int(current)
else:
    current_val = 200       

# If n is greater than the current value, update the shuffle partitions setting
if n > current_val:            
    spark.conf.set("spark.sql.shuffle.partitions", str(n))

### Models
Let's configure a list of models we are going to apply to our time series for evaluation and forecasting. A comprehensive list of all supported models is available in [mmf_sa/models/README.md](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/README.md). Look for the models where `model_type: local`; these are the local models we import from [statsforecast](https://github.com/Nixtla/statsforecast) and [sktime](https://github.com/sktime/sktime). Check their documentations for the description of each model. 

In [0]:
active_models = [
    "StatsForecastBaselineWindowAverage",
    "StatsForecastBaselineSeasonalWindowAverage",
    "StatsForecastBaselineNaive",
    "StatsForecastBaselineSeasonalNaive",
    "StatsForecastAutoArima",
    "StatsForecastAutoETS",
    "StatsForecastAutoCES",
    "StatsForecastAutoTheta",
    "StatsForecastAutoTbats",
    "StatsForecastAutoMfles",
    "StatsForecastTSB",
    "StatsForecastADIDA",
    "StatsForecastIMAPA",
    "StatsForecastCrostonClassic",
    "StatsForecastCrostonOptimized",
    "StatsForecastCrostonSBA",
    "SKTimeProphet",
    ]

### Run MMF

Now, we can run the evaluation and forecasting using `run_forecast` function defined in [mmf_sa/models/__init__.py](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/__init__.py). Make sure to set `freq="W"` in `run_forecast` function.

In [0]:
run_forecast(
    spark=spark,
    train_data=f"{catalog}.{db}.m4_weekly_train",
    scoring_data=f"{catalog}.{db}.m4_weekly_train",
    scoring_output=f"{catalog}.{db}.weekly_scoring_output",
    evaluation_output=f"{catalog}.{db}.weekly_evaluation_output",
    group_id="unique_id",
    date_col="ds",
    target="y",
    freq="W",
    prediction_length=4,
    backtest_length=12,
    stride=1,
    metric="smape",
    train_predict_ratio=1,
    data_quality_check=True,
    resample=False,
    active_models=active_models,
    experiment_path=f"/Users/{user}/mmf/m4_weekly",
    use_case_name="m4_weekly",
)

### Evaluate
In `evaluation_output` table, the we store all evaluation results for all backtesting trials from all models.

In [0]:
display(
  spark.sql(f"""
    select * from {catalog}.{db}.weekly_evaluation_output 
    where unique_id = 'W1'
    order by unique_id, model, backtest_window_start_date
    """))

### Forecast
In `scoring_output` table, forecasts for each time series from each model are stored.

In [0]:
display(spark.sql(f"""
                  select * from {catalog}.{db}.weekly_scoring_output 
                  where unique_id = 'W1'
                  order by unique_id, model, ds
                  """))

Refer to the [notebook](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/examples/post-evaluation-analysis.ipynb) for guidance on performing fine-grained model selection after running `run_forecast`.

### Delete Tables
Let's clean up the tables.

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.weekly_evaluation_output"))

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.weekly_scoring_output"))